In [28]:
pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
import networkx as nx
from pyvis.network import Network


In [30]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [31]:
import nest_asyncio

nest_asyncio.apply()

In [32]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Set LLM (OpenAI)

In [33]:
load_dotenv()  # Charge les variables depuis le fichier .env
api_key = os.getenv("OPENAI_API_KEY_UP")
# Modifier ou ajouter une variable d'environnement
os.environ["OPENAI_API_KEY"] = api_key

In [34]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings

In [35]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5


INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Local)

In [36]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [37]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
assistant: As a data governance consultant, I don't have personal preferences or feelings. However, I can provide insights into popular data tools that are widely used in the industry. Some of the most favored tools for data governance include:

1. **Collibra**: Known for its comprehensive data governance capabilities, including data cataloging, data stewardship, and policy management.

2. **Informatica**: Offers a robust suite of data governance tools, including data quality, data cataloging, and master data management.

3. **Alation**: Focuses on data cataloging and collaboration, helping organizations improve data discovery and governance.

4. **Talend**: Provides data integration and data quality tools that support data governance init

# Load storage contexts

## Load vector storage context

In [38]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

## Load knowledge graph storage context

In [39]:
graph_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="knowledge_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="knowledge_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="knowledge_graph"),
)

## Load onto graph storage context


In [40]:
onto_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="onto_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="onto_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="onto_graph"),
)

In [41]:
print(onto_storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x2c0bbc410>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x2c0868490>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={})), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x2bdaf54d0>, property_graph_store=None)


# Load index

## Load vector index

In [42]:
simple_index = load_index_from_storage(vector_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


### Set retriever

In [43]:
simple_query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [44]:
simple_rag_retriever = simple_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = simple_query_engine.query(
    "Qui est-tu ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [45]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, voici un plan structuré :

1. **Compréhension des Données**
   - Identifier les sources de données disponibles (historique des prêts, données démographiques, etc.).
   - Définir l'objectif de la prédiction : comprendre les facteurs influençant le défaut de paiement.

2. **Nettoyage des Données**
   - Supprimer ou corriger les données inexactes ou les doublons.
   - Remplir les données manquantes et uniformiser les formats (par exemple, dates).

3. **Exploration des Données**
   - Analyser les tendances et les relations entre les variables.
   - Utiliser des visualisations pour identifier des patterns ou anomalies.

4. **Préparation des Données**
   - Transformer les données non structurées en un format exploitable.
   - Sélectionner les caractéristiques pertinentes pour la modélisation.

5. **Choix du Modèle**
   - Sélectionner des modèles de machine learning adaptés (régression logistique, arbres de décision, etc.).
   - Considérer l'utilisation d'outils spécialisés pour traiter les données complexes.

6. **Entraînement et Validation du Modèle**
   - Diviser les données en ensembles d'entraînement et de test.
   - Entraîner le modèle sur l'ensemble d'entraînement et valider sa performance sur l'ensemble de test.

7. **Évaluation du Modèle**
   - Utiliser des métriques de performance (précision, rappel, F1-score) pour évaluer le modèle.
   - Tester avec des parties prenantes pour recueillir des retours et apporter des modifications si nécessaires.

8. **Interprétation et Communication des Résultats**
   - Créer des visualisations claires pour communiquer les résultats aux parties prenantes.
   - Assurer que les visualisations sont compréhensibles et utilisables par tous, y compris les personnes avec un handicap.

9. **Mise en Production et Surveillance**
   - Déployer le modèle dans un environnement de production.
   - Surveiller la performance du modèle et effectuer des mises à jour régulières si nécessaire.

Ce plan permet de structurer le processus de prédiction de défaut de paiement en utilisant une approche méthodique et centrée sur les données.</b>

## Load graph index

In [46]:
graph_index = load_index_from_storage(graph_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


In [47]:
nx_graph = graph_index.get_networkx_graph()

In [48]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 49


In [49]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 74 nodes and 49 edges
Nodes: ['Exactitude', 'Justes et cohérentes', 'Complétude', 'Complet', 'Nettoyage', 'Erreurs', 'Normalisation', 'Formats', 'Dataviz project', 'Relation entre inputs et visualisations de données', 'Bibliothèque', 'Description', '3v', 'Volume', 'Vélocité', 'Base de données relationnelles', 'Données structurées', 'Outils comme sql', 'Base de données non relationnelles', 'Données non structurées', 'Tirer', 'Données', 'Exploitables', 'Format prédéfini', 'Texte libre', 'Claire et standardisée', 'Passer', 'Interprétation des tendances', 'La data visualisation', 'Données brutes en graphiques', 'Information claire', 'Processus', 'Visualisation', 'Objectif', 'Message', 'Théophile', 'Outil informatique', 'Ordinateur', 'Quantité de données phénoménales', 'Phase', 'Choix des visualisations', 'Les utilisateurs', 'Les données', 'La visualisation', 'La source', 'Traitement', 'Les dataviz', 'Tester', 'Parties prenantes', 'Utilisateur', "Niveaux d'informations", 'Tout le

### Set retriever

In [50]:
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [ ]:
graph_rag_retriever = graph_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = graph_query_engine.query(
    "Qui est-tu ?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: a5d8f92d-8c56-4951-9949-000a7f8e5928: Ajout d'éléments dynamiques pour permettre aux [[utilisateur]]s d'explorer le...
> Querying with idx: a5d8f92d-8c56-4951-9949-000a7f8e5928: Ajout d'éléments dynamiques pour permettre aux [[utilisateur]]s d'explorer le...
> Querying with idx: a5d8f92d-8c56-4951-9949-000a7f8e5928: Ajout d'éléments dynamiques pour permettre aux [[utilisateur]]s d'explorer le...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 6b6f94f9-b344-436a-941e-a0dfe3ea3d0b: Réflexion à mener sur la figure de l'utilisateur dans le parcours par rapport...
> Querying with idx: 6b6f94f9-b344-436a-941e-a0dfe3ea3d0b: Réflexion à mener sur la figure de l'utilisateur dans le parcours par rapport...
> Querying with idx: 6b6f94f9-b344-436a-941e-a0dfe3ea3d0b: Réflexion à mener sur la figure de l'utilisateur dans le parcours par rapport...
INFO:llama_index.core.indices.knowledge_graph.

In [52]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, il est essentiel de suivre un plan structuré qui inclut plusieurs étapes clés. Voici un plan détaillé :

1. **Collecte et Préparation des Données :**
   - Rassembler les données pertinentes sur les clients, y compris les historiques de crédit, les revenus, les dettes, et d'autres facteurs financiers.
   - S'assurer que les données sont organisées de manière claire et standardisée, souvent dans un tableau avec des lignes et des colonnes.
   - Nettoyer les données pour éliminer les anomalies et les valeurs manquantes.

2. **Exploration des Données :**
   - Analyser la structure des données pour comprendre les relations entre les différentes variables.
   - Utiliser des outils comme SQL pour interroger les données structurées et obtenir des insights préliminaires.

3. **Analyse des Données :**
   - Identifier les tendances et les corrélations qui pourraient indiquer un risque de défaut.
   - Utiliser des visualisations appropriées pour représenter les données, comme des diagrammes à barres pour les comparaisons entre catégories ou des courbes pour les tendances.

4. **Sélection des Caractéristiques :**
   - Choisir les variables les plus pertinentes qui influencent le risque de défaut, en se basant sur l'analyse exploratoire.

5. **Modélisation Prédictive :**
   - Utiliser des algorithmes de machine learning pour construire un modèle prédictif. Les modèles couramment utilisés incluent la régression logistique, les arbres de décision, ou les forêts aléatoires.
   - Diviser les données en ensembles d'entraînement et de test pour évaluer la performance du modèle.

6. **Évaluation du Modèle :**
   - Mesurer la précision du modèle en utilisant des métriques telles que l'accuracy, le rappel, et la précision.
   - Ajuster le modèle en fonction des résultats pour améliorer sa performance.

7. **Interprétation et Communication des Résultats :**
   - Interpréter les résultats du modèle pour comprendre les facteurs clés qui contribuent au risque de défaut.
   - Communiquer les résultats de manière compréhensible et utilisable par tous les utilisateurs, y compris ceux avec un handicap.

8. **Mise en Œuvre et Surveillance :**
   - Déployer le modèle dans un environnement de production pour prédire les défauts en temps réel.
   - Surveiller la performance du modèle et le mettre à jour régulièrement pour s'adapter aux nouvelles données et aux changements de comportement des clients.

Ce plan permet de structurer le processus de prédiction du défaut de paiement de manière méthodique et efficace.</b>

## Load onto index

In [53]:
onto_storage_context = StorageContext.from_defaults(persist_dir="onto_graph")
# Load the PropertyGraphIndex from the storage context

onto_index = load_index_from_storage(onto_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


In [54]:
onto_index.property_graph_store.save_networkx_graph(
    name="OntoGraph.html"
)

### Set retriever

In [55]:
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=8,
)

### Test retriever

In [ ]:
onto_rag_retriever = onto_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = onto_query_engine.query(
    "Qui est-tu ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [57]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, voici un plan structuré :

1. **Collecte et Préparation des Données**
   - **Collecte des Données** : Rassembler les données historiques des clients, y compris les informations démographiques, les antécédents de crédit, les transactions bancaires, etc.
   - **Nettoyage des Données** : Supprimer ou corriger les données inexactes, éliminer les doublons, et remplir les données manquantes.
   - **Normalisation et Transformation** : Uniformiser les formats de données et transformer les données pour les adapter aux outils d'analyse.

2. **Exploration et Analyse des Données**
   - **Analyse Exploratoire** : Identifier les tendances et les relations dans les données à l'aide de visualisations.
   - **Sélection des Caractéristiques** : Déterminer les variables les plus pertinentes qui influencent le défaut de paiement.

3. **Modélisation Prédictive**
   - **Choix du Modèle** : Sélectionner un modèle de machine learning approprié (par exemple, régression logistique, arbres de décision, forêts aléatoires).
   - **Entraînement du Modèle** : Utiliser un ensemble de données d'entraînement pour ajuster le modèle.
   - **Validation du Modèle** : Évaluer la performance du modèle avec un ensemble de données de validation.

4. **Évaluation et Optimisation**
   - **Évaluation des Performances** : Utiliser des métriques telles que l'accuracy, le rappel, et la précision pour évaluer le modèle.
   - **Optimisation** : Ajuster les hyperparamètres et améliorer le modèle en fonction des résultats d'évaluation.

5. **Déploiement et Surveillance**
   - **Déploiement** : Intégrer le modèle dans le système bancaire pour des prédictions en temps réel.
   - **Surveillance Continue** : Suivre la performance du modèle et mettre à jour les données et le modèle régulièrement pour maintenir l'exactitude des prédictions.

6. **Interprétation et Communication**
   - **Interprétation des Résultats** : Analyser les résultats pour comprendre les facteurs clés de défaut.
   - **Communication** : Présenter les résultats aux parties prenantes à l'aide de visualisations claires et concises.

Ce plan permet de structurer le processus de prédiction du défaut de paiement en utilisant des méthodes analytiques et des outils de dataviz pour une meilleure compréhension et application des résultats.</b>

# Visualize knowledge graph

In [58]:
nx_graph = graph_index.get_networkx_graph()

In [59]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 49


In [60]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 74 nodes and 49 edges
Nodes: ['Exactitude', 'Justes et cohérentes', 'Complétude', 'Complet', 'Nettoyage', 'Erreurs', 'Normalisation', 'Formats', 'Dataviz project', 'Relation entre inputs et visualisations de données', 'Bibliothèque', 'Description', '3v', 'Volume', 'Vélocité', 'Base de données relationnelles', 'Données structurées', 'Outils comme sql', 'Base de données non relationnelles', 'Données non structurées', 'Tirer', 'Données', 'Exploitables', 'Format prédéfini', 'Texte libre', 'Claire et standardisée', 'Passer', 'Interprétation des tendances', 'La data visualisation', 'Données brutes en graphiques', 'Information claire', 'Processus', 'Visualisation', 'Objectif', 'Message', 'Théophile', 'Outil informatique', 'Ordinateur', 'Quantité de données phénoménales', 'Phase', 'Choix des visualisations', 'Les utilisateurs', 'Les données', 'La visualisation', 'La source', 'Traitement', 'Les dataviz', 'Tester', 'Parties prenantes', 'Utilisateur', "Niveaux d'informations", 'Tout le

# (Simple) Query the vectors

In [61]:
query = "Qui est-tu ?"
query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [62]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, voici un plan structuré :

1. **Compréhension des Données**
   - Identifier les sources de données disponibles (historique des prêts, données démographiques, etc.).
   - Définir l'objectif de la prédiction : comprendre les facteurs influençant le défaut de paiement.

2. **Nettoyage des Données**
   - Supprimer ou corriger les données inexactes ou les doublons.
   - Remplir les données manquantes et uniformiser les formats (par exemple, dates).

3. **Exploration des Données**
   - Analyser les données pour identifier des tendances ou des corrélations.
   - Utiliser des visualisations pour mieux comprendre les relations entre les variables.

4. **Préparation des Données**
   - Transformer les données non structurées en un format exploitable.
   - Sélectionner les caractéristiques pertinentes pour la modélisation.

5. **Choix du Modèle**
   - Sélectionner un modèle de machine learning approprié (par exemple, régression logistique, arbres de décision).
   - Justifier le choix du modèle en fonction des données et de l'objectif.

6. **Entraînement du Modèle**
   - Diviser les données en ensembles d'entraînement et de test.
   - Entraîner le modèle sur l'ensemble d'entraînement.

7. **Évaluation du Modèle**
   - Tester le modèle sur l'ensemble de test.
   - Utiliser des métriques d'évaluation (précision, rappel, F1-score) pour mesurer la performance.

8. **Optimisation du Modèle**
   - Ajuster les hyperparamètres pour améliorer la performance.
   - Réévaluer le modèle après chaque ajustement.

9. **Interprétation des Résultats**
   - Analyser les résultats pour comprendre les facteurs clés du défaut de paiement.
   - Communiquer les résultats de manière claire et compréhensible.

10. **Mise en Production**
    - Déployer le modèle dans un environnement de production.
    - Mettre en place un système de surveillance pour suivre la performance du modèle.

11. **Amélioration Continue**
    - Recueillir des retours et ajuster le modèle en fonction des nouvelles données.
    - Réévaluer régulièrement le modèle pour s'assurer de sa pertinence et de son efficacité.</b>

# (Simple) Query the knowledge graph 

In [ ]:
query = "Qui est-tu ?"
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: a5d8f92d-8c56-4951-9949-000a7f8e5928: Ajout d'éléments dynamiques pour permettre aux [[utilisateur]]s d'explorer le...
> Querying with idx: a5d8f92d-8c56-4951-9949-000a7f8e5928: Ajout d'éléments dynamiques pour permettre aux [[utilisateur]]s d'explorer le...
> Querying with idx: a5d8f92d-8c56-4951-9949-000a7f8e5928: Ajout d'éléments dynamiques pour permettre aux [[utilisateur]]s d'explorer le...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 6b6f94f9-b344-436a-941e-a0dfe3ea3d0b: Réflexion à mener sur la figure de l'utilisateur dans le parcours par rapport...
> Querying with idx: 6b6f94f9-b344-436a-941e-a0dfe3ea3d0b: Réflexion à mener sur la figure de l'utilisateur dans le parcours par rapport...
> Querying with idx: 6b6f94f9-b344-436a-941e-a0dfe3ea3d0b: Réflexion à mener sur la figure de l'utilisateur dans le parcours par rapport...
INFO:llama_index.core.indices.knowledge_graph.

In [64]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, il est essentiel de suivre un plan structuré qui inclut plusieurs étapes clés. Voici un plan détaillé :

1. **Collecte et Préparation des Données :**
   - Rassembler les données pertinentes sur les clients, y compris les historiques de crédit, les revenus, les dettes, et d'autres facteurs financiers.
   - Nettoyer les données pour éliminer les valeurs manquantes ou aberrantes.
   - Organiser les données de manière claire et standardisée, souvent dans un tableau avec des lignes et des colonnes.

2. **Exploration des Données :**
   - Analyser la structure des données pour comprendre les relations entre les différentes variables.
   - Utiliser des outils de visualisation pour identifier les tendances et les anomalies.

3. **Sélection des Caractéristiques :**
   - Identifier les caractéristiques les plus pertinentes qui pourraient influencer le défaut de paiement.
   - Utiliser des techniques de réduction de dimensionnalité si nécessaire pour simplifier le modèle.

4. **Choix du Modèle :**
   - Sélectionner un modèle de machine learning approprié, tel que la régression logistique, les arbres de décision, ou les forêts aléatoires.
   - Considérer l'utilisation de modèles plus avancés comme les réseaux de neurones si les données sont suffisamment volumineuses et complexes.

5. **Entraînement du Modèle :**
   - Diviser les données en ensembles d'entraînement et de test.
   - Entraîner le modèle sur l'ensemble d'entraînement en ajustant les paramètres pour optimiser la performance.

6. **Évaluation du Modèle :**
   - Tester le modèle sur l'ensemble de test pour évaluer sa précision et sa capacité à prédire les défauts de paiement.
   - Utiliser des métriques telles que l'accuracy, le rappel, et la précision pour évaluer la performance.

7. **Optimisation et Validation :**
   - Affiner le modèle en ajustant les hyperparamètres et en utilisant des techniques de validation croisée.
   - S'assurer que le modèle n'est pas surajusté et qu'il généralise bien aux nouvelles données.

8. **Déploiement et Surveillance :**
   - Déployer le modèle dans un environnement de production pour prédire les défauts de paiement en temps réel.
   - Mettre en place un système de surveillance pour suivre la performance du modèle et effectuer des mises à jour si nécessaire.

En suivant ces étapes, vous pouvez développer un modèle robuste pour prédire les défauts de paiement sur les prêts bancaires.</b>

# (Simple) Query the onto graph 

In [65]:
query = "Qui est-tu ?"
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=2,
)

response = onto_query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [66]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, voici un plan structuré :

1. **Collecte des données :**
   - Rassembler des données historiques sur les clients, y compris les informations démographiques, les antécédents de crédit, les revenus, les dettes existantes, etc.
   - Inclure des données sur les prêts précédents et leur statut (remboursé ou en défaut).

2. **Préparation des données :**
   - Nettoyer les données pour éliminer les valeurs manquantes ou aberrantes.
   - Normaliser ou standardiser les variables numériques pour assurer une échelle uniforme.
   - Encoder les variables catégorielles en valeurs numériques.

3. **Exploration des données :**
   - Utiliser des techniques de visualisation de données pour identifier les tendances et les relations entre les variables.
   - Analyser les inférences pour comprendre les facteurs qui influencent le défaut de paiement.

4. **Sélection des caractéristiques :**
   - Identifier les caractéristiques les plus pertinentes qui influencent le défaut de paiement à l'aide de techniques comme l'analyse de corrélation ou l'importance des caractéristiques.

5. **Modélisation :**
   - Choisir un modèle de machine learning approprié, tel que la régression logistique, les arbres de décision, ou les forêts aléatoires.
   - Diviser les données en ensembles d'entraînement et de test.

6. **Entraînement du modèle :**
   - Entraîner le modèle sur l'ensemble d'entraînement.
   - Ajuster les hyperparamètres pour optimiser les performances du modèle.

7. **Évaluation du modèle :**
   - Évaluer le modèle sur l'ensemble de test en utilisant des métriques telles que l'exactitude, la précision, le rappel, et le score F1.
   - Analyser les erreurs pour comprendre les limitations du modèle.

8. **Interprétation et déploiement :**
   - Interpréter les résultats pour fournir des insights exploitables aux décideurs.
   - Déployer le modèle dans un environnement de production pour prédire les défauts de paiement en temps réel.

9. **Surveillance et mise à jour :**
   - Surveiller les performances du modèle au fil du temps.
   - Mettre à jour le modèle avec de nouvelles données pour maintenir sa précision et sa pertinence. 

Ce plan permet de structurer l'approche pour prédire les défauts de paiement en utilisant des données et des techniques analytiques avancées.</b>

## (Node retriever)

In [67]:
retriever = onto_index.as_retriever(
    include_text=True,  # include source text, default True
)

nodes = retriever.retrieve("Qui est-tu ?")


for node in nodes:
    print(node.text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Here are some facts extracted from the provided text:

jeu de données -> CONFRONTED_WITH -> que faire

Réflexion à mener sur la figure de l'utilisateur dans le parcours par rapport aux données ? 

Retrouver le schéma dataviz : que faire si je me retrouve face à un jeu de données 
Est ce que c'est quelque chose que le LLM peut lire ?
Here are some facts extracted from the provided text:

pertinence -> EXTENDS_TO -> besoins du client

Quels critères pour dire que des données sont de qualité ?
- Exactitude : les données sont justes et cohérentes
- Complétude : le jeu de données est complet 
- Pertinence : les données que l'on a sont elles pertinentes pour répondre à nos besoins (et par extension ceux du client ?)

Des données de bonnes qualit

# Have a real chat with your data

## Set up the engines

### Vector engines

In [68]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        " "
        " "
        "."
    ),
    verbose=False,
)

### Graph engines

In [69]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    verbose=False,
)

### Onto engines

In [70]:
#onto_chat_engine = onto_index.query_engine(
#    chat_mode="condense_plus_context",
#    llm=llm
#)

## Generate

In [71]:
graph_chat_engine.reset()

In [72]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Tu dois générer des générer des méthodologies de dataviz centrées autour de la découverte de lignes de force entre idées"
        " "
        " "
    ),
    verbose=False,
)

In [75]:
response_stream = graph_chat_engine.stream_chat(""" """)

INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question:  
Condensed question:  
Condensed question:  
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 02d20359-7216-420e-bdc7-6fe57a70b452: Art de raconter une histoire avec des [[données]] 
Implique de : 
- Sélection...
> Querying with idx: 02d20359-7216-420e-bdc7-6fe57a70b452: Art de raconter une histoire avec des [[données]] 
Implique de : 
- Sélection...
> Querying with idx: 02d20359-7216-420e-bdc7-6fe57a70b452: Art de raconter une histoire avec des [[données]] 
Implique de : 
- Sélection...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: a5d8f92d-8c56-4951-9949-000a7f8e5928: Ajout d'éléments dynamiques pour permettre aux [[utilisateur]]s d'explorer le...
> Querying with idx: a5d8f92d-8c56-4951-9949-000a7f8e5928: Ajout d'éléments dynamiques pour permettre aux [[utilisateur]]s d'explorer le...
> Querying with idx: a5d8f92d-8c56-4951-9949-000a7f8e5928: Ajout d'éléments dynamiques pour permettre aux [[utilisateur]]s d'explorer le...
INFO:llama_index.core.indices.knowledge_graph.

In [76]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Hello! How can I assist you today?

In [77]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

In [78]:
section_modele = """ 


"""

In [79]:
response_stream = graph_chat_engine.stream_chat("""Imite le style d'écriture de la {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Pouvez-vous imiter le style d'écriture de la {section_modele} ?
Condensed question: Pouvez-vous imiter le style d'écriture de la {section_modele} ?
Condensed question: Pouvez-vous imiter le style d'écriture de la {section_modele} ?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 5a8ae5a3-e79f-4ca5-b9ed-4554c57f7e2f: ---
partie de:
  - "[[Dataviz Project]]"
  - "[[dataviz]]"
---
A chaque visua...
> Querying with idx: 5a8ae5a3-e79f-4ca5-b9ed-4554c57f7e2f: ---
partie de:
  - "[[Dataviz Project]]"
  - "[[dataviz]]"
---
A chaque visua...
> Querying with idx: 5a8ae5a3-e79f-4ca5-b9ed-4554c57f7e2f: ---
partie de:
  - "[[Dataviz Project]]"
  - "[[dataviz]]"
---
A chaque visua...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 02d20359-7216-420e-bdc7-6fe57a70b452: Art de raconter une histoire avec des [[données]] 
Implique de : 
- Sélection...
> Querying with idx: 02d20359-7216-420e-bdc7-6fe57a70b452: Art de raconter une histoire avec des [[données]] 
Implique de : 
- Sélection...
> Querying with idx: 02d20359-7216-420e-bdc7-6fe57a70b452: Art de raconter une histoire avec des [[données]] 
Implique de : 
- Sélection...
INFO:llama_index.core.indices.knowledge_graph.

In [80]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Je suis désolé, mais il semble qu'il y ait eu une petite confusion dans votre demande. Pourriez-vous préciser quel style d'écriture ou quel modèle vous aimeriez que j'imite ? Cela m'aiderait à mieux répondre à votre demande. Merci !

In [81]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

## Sum-up

In [85]:
chat_history = memory.get_all()

# Assuming chat_history is available and contains your messages
assistant_messages = [
    message.content 
    for message in chat_history 
    if message.role == MessageRole.ASSISTANT  # Compare with the enum directly
]


output_filename = r"/Users/arthursarazin/Documents/coreandgraphs/graphandvisualize/output.md"
# Write to a Markdown file
with open(output_filename, "w", encoding="utf-8") as f:
    for msg in assistant_messages:
        f.write(msg + "\n\n") 